In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

TOKEN = os.environ.get('TOKEN')
HOST = os.environ.get('HOST')
MODEL = os.environ.get('MODEL')

llm_client = OpenAI(
  api_key=TOKEN,
  base_url=HOST
)

chat_completion = llm_client.chat.completions.create(
  messages=[
    {"role": "system", "content": "You are an AI assistant"},
    {"role": "user", "content": "Tell me about Large Language Models"}
  ],
  model=MODEL,
  max_tokens=256
)

print(chat_completion.choices[0].message.content)


In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder
import chromadb

# Load embedding and cross-encoder models
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Initialize ChromaDB client
client = chromadb.PersistentClient(path="knowledge_base")
collection = client.get_or_create_collection(name="gen_ai_crash_course_collection")

def fetch_top_relevant_queries(query, collection, top_k=10):
    query_embedding = embedding_model.encode(query).tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return results['documents'][0], results['metadatas'][0], results['distances'][0]

def retrieve_relevant_content(query):
    documents, metadata, scores = fetch_top_relevant_queries(query, collection, top_k=10)
    # Compute cross-encoder scores
    scored_queries = []
    for doc, meta, score in zip(documents, metadata, scores):
        doc_text = doc.replace("\n", " ")
        cross_score = cross_encoder.predict([(query, doc_text)])[0]
        scored_queries.append({"document": doc_text, "metadata": meta, "similarity_score": score, "cross_encoder_score": cross_score})

    # Sort by cross-encoder score and take top 3
    top_docs = sorted(scored_queries, key=lambda x: x['cross_encoder_score'], reverse=True)[:3]

    # Combine top 3 documents into a single paragraph
    combined_text = " ".join([item['document'] for item in top_docs])

    return combined_text


retrieve_relevant_content("What is l and t doing for sustainability of environemtn?")

In [ ]:
def enhance_prompt(query):
    """Enhances the user query to improve retrieval accuracy."""
    chat_completion = llm_client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an AI assistant for L&T group. Your task is to refine user queries for better information retrieval. Make the query clearer, more specific, and structured to help retrieve relevant documents accurately."},
            {"role": "user", "content": f"Enhance this query for better retrieval: {query}"}
        ],
        model=MODEL,
        max_tokens=50
    )
    return chat_completion.choices[0].message.content.strip()

def enhanced_llm_call(user_query):
    """Enhances query, retrieves relevant content, and gets final response."""
    
    # Step 1: Enhance the query for better retrieval
    refined_query = enhance_prompt(user_query)

    # Step 2: Retrieve relevant content using the enhanced query
    relevant_content = retrieve_relevant_content(refined_query)

    # Step 3: Send both the retrieved content and the original user question to LLM
    chat_completion = llm_client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an AI assistant. Answer the user's question using the given context."},
            {"role": "user", "content": f"Context:\n{relevant_content}\n\nUser Question: {user_query}"}
        ],
        model=MODEL,
        max_tokens=512
    )

    return chat_completion.choices[0].message.content

# Example Usage
user_query = "Who is your chairman"
final_output = enhanced_llm_call(user_query)
print(final_output)
